# Computer Vision on the Descartes Labs Platform - Training a Simple UNet Model
__________________

This notebook will demonstrate how one can utilize Descartes Labs Workbench to efficiently prototype and iterate on training an image segmentation model. This is meant to serve _solely as a jumping off point_ and is not intended to be used as a panacea for all machine learning needs. 

#### *__Note: It is STRONGLY urged that you use a GPU-enabled environment to execute this notebook__*

The general outline of this sample is as follows:
* Load in the training data generated in [03a Generate Training Data.ipynb](03a%20Generate%20Training%20Data.ipynb), saved locally as .npy files
* Train and compile a simple convolutional neural network with a U-Net architecture using [TensorFlow](https://www.tensorflow.org/guide/keras) 
    * *Note: This should not be seen as an end-all deep learning model, this is an extremely simplified example focused on the general patterns over accuracy*
* Save our trained segmentation model as a [`Blob`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob) to infer asynchronously using [`Batch Compute`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html) in [03c Deploying a Segmentation Model.ipynb](03c%20Deploying%20a%20Segmentation%20Model.ipynb)

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Blob, properties as p

In [ ]:
import pickle, os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    UpSampling2D,
    Concatenate,
)
from tensorflow.keras.models import Model

## Data Preparation
Load in our data and mask arrays from the previous notebook:

In [ ]:
data_array = np.load("data_array.npy")
mask_array = np.load("mask_array.npy")
data_array.shape, mask_array.shape

Plot a single pair:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(data_array[0] / 255)
ax[1].imshow(mask_array[0, :, :, 0])

Very simply 80:20 train/test split:

In [ ]:
train_data = data_array[:800]
val_data = data_array[800:]
train_masks = mask_array[:800]
val_masks = mask_array[800:]

In [ ]:
train_data.shape, val_data.shape

In [ ]:
train_masks.shape, val_masks.shape

## Training a Computer Vision Model
A sample U-Net architecture, commonly used in image segmentation tasks. In practice this is where you should input your own methodology and iterate!

In [ ]:
class CustomUNet:
    def __init__(
        self, optimizer="adam", activation="sigmoid", loss="binary_crossentropy"
    ):
        self.optimizer = optimizer
        self.activation = activation
        self.loss = loss
        self.model = self.build_model()

    def build_model(self):
        # Define input layer
        inputs = Input(shape=(None, None, 3))

        # Encoder
        conv1 = Conv2D(64, 3, activation="relu", padding="same")(inputs)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        conv2 = Conv2D(128, 3, activation="relu", padding="same")(pool1)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        conv3 = Conv2D(256, 3, activation="relu", padding="same")(pool2)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        conv4 = Conv2D(512, 3, activation="relu", padding="same")(pool3)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        # Center
        conv5 = Conv2D(1024, 3, activation="relu", padding="same")(pool4)

        # Decoder
        up6 = Concatenate()([UpSampling2D(size=(2, 2))(conv5), conv4])
        conv6 = Conv2D(512, 3, activation="relu", padding="same")(up6)
        up7 = Concatenate()([UpSampling2D(size=(2, 2))(conv6), conv3])
        conv7 = Conv2D(256, 3, activation="relu", padding="same")(up7)
        up8 = Concatenate()([UpSampling2D(size=(2, 2))(conv7), conv2])
        conv8 = Conv2D(128, 3, activation="relu", padding="same")(up8)
        up9 = Concatenate()([UpSampling2D(size=(2, 2))(conv8), conv1])
        conv9 = Conv2D(64, 3, activation="relu", padding="same")(up9)

        # Output layer
        outputs = Conv2D(1, 1, activation=self.activation)(conv9)

        # Define the model
        model = Model(inputs=inputs, outputs=outputs)

        # Compile the model
        model.compile(optimizer=self.optimizer, loss=self.loss, metrics=["accuracy"])

        return model


# Example usage:
unet_model = CustomUNet()
model = unet_model.model

### _Note: This will take extremely long if you are not using a GPU-enabled environment!_

Here we fit our model. On the GPU-enabled Descartes Labs Workbench this step takes roughly 35 minutes to complete.

In [ ]:
model.fit(train_data, train_masks, epochs=100, validation_data=(val_data, val_masks))

In [ ]:
model.summary()

## Testing Predictions
Inferring our model over our validation data:

In [ ]:
preds = model.predict(val_data)

And visualizing one result:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(val_data[0] / 255)
ax[1].imshow(preds[0, :, :, 0])

## Saving for Scale
At this point, you should again be iterating in the real world. Assuming we are happy with the performance of our model, we can next save it as a static file:

In [ ]:
model.save("training_segmentation.keras")

Here we save as a blob to reference in [03c Deploying a Segmentation Model.ipynb](03c%20Deploying%20a%20Segmentation%20Model.ipynb):

In [ ]:
org = dl.auth.Auth().payload["org"]
user_hash = dl.auth.Auth().namespace
namespace = f"{org}:{user_hash}" if org else user_hash
namespace

Note, you don't always need to delete and overwrite your blobs such as in the following cell. This is an example where we do not care about deleting previous iterations. In practice you should try to preserve, attribute, and version control your model weights files for iteration!

In [ ]:
try:
    # Create a new Blob object
    blob = Blob(
        name="training_segmentation",
        tags=["deep-learning-examples"],
    )
    # Upload our model to this Blob:
    blob.upload("training_segmentation.keras")
except:
    print("Blob already exists, overwriting")
    # Already exists, overwriting
    blob = Blob.get(name="training_segmentation", 
                    namespace=namespace
                   )
    blob.delete()
    print("Deleted blob")
    # Create a new Blob object
    blob = Blob(
        name="training_segmentation",
        tags=["deep-learning-examples"],
    )
    # Upload our model to this Blob:
    blob.upload("training_segmentation.keras")
blob

Finally cleaning up:

In [ ]:
os.remove("training_segmentation.keras")

#### Note: After completing this notebook it is recommended to shut your notebook kernel down before moving on to [03c Deploying a Segmentation Model.ipynb](03c%20Deploying%20a%20Segmentation%20Model.ipynb)